## The Team - Les Perchistes 🌴🦧

- Marin Philipe $\Longrightarrow$ Robot control 👨‍💻
- Neil Chadli $\Longrightarrow$ Path finding 🔍
- Loïc Delineau $\Longrightarrow$ Computer Vision 🕵️
- Matthieu Scharffe $\Longrightarrow$ Kalman Filter 🤞

# Introduction

## Project Description

The objective of this project is to navigate the Thymio robot from a starting point to a designated endpoint, overcoming obstacles represented by black shapes along its path. The robot must also be capable of responding to unexpected obstacles that appear in its trajectory. An overhead camera will be used to calculate the robot's path and determine its location. Additionally, we aim to ensure that the Kalman filter can accurately localize the robot both with and without the camera.

# Setup

## Play area structure

**Talk about the structure and show images**

## Software

The code is entirely written in python. The libraries used are the following :

- tdmclient
- Numpy
- OpenCV
- **Complete the list**

In [ ]:
### Importing the libraries

import numpy as np
import matplotlib.pyplot as plt

## Measurements

The measurements used to localized the thymio come from the computer vision and the speed sensors of the robot.

## Parametrization of the problem

![Parametrization of the problem](./img/repère_pb.jpeg)

The studied problem is composed of 2 basis and 3 frames. The main frame $\mathcal{R}_0 = (O,\overrightarrow{x_0}, \overrightarrow{y_0}, \overrightarrow{z_0})$ is defined as so : 

- The origin of the frame $(O)$ is in the top left corner of the image.
- The axis $(O,\overrightarrow{x_0})$ goes from left to right over the upper border.
- The axis $(O,\overrightarrow{y_0})$ goes from top to bottom over the left border.
- The axis $(O,\overrightarrow{z_0})$ points toward the ground.

Let's define the point $T$ as the middle point of the wheels. $I_g$ (resp. $I_d$) is the contact point of the left wheel (resp. right). $L$ is the distance between $T$ and $I_g$ or $I_d$. We can define the second frame $\mathcal{R}_2 = (T,\overrightarrow{x_1}, \overrightarrow{y_1}, \overrightarrow{z_0})$.

- The axis $(T,\overrightarrow{x_1})$ is along the median of $[I_g,I_d]$ toward the front of the robot. 
- The axis $(T,\overrightarrow{y_1})$ goes from $T$ to $I_d$.


Let's define the third frame where some derivations will be made : $\mathcal{R}_1 = (0,\overrightarrow{x_1}, \overrightarrow{y_1}, \overrightarrow{z_0})$. $\theta$ is defined as rotation angle between basis 0 and basis 1. It is important to note that to go from basis 0 to 1, the rotation matrix is :

$$
    \begin{pmatrix} x_1\\ y_1 \end{pmatrix} = \underbrace{\begin{pmatrix} \cos{\theta} & \sin{\theta}\\ -\sin{\theta} & \cos{\theta} \end{pmatrix}}_{R(\theta)} \begin{pmatrix} x_0\\ y_0 \end{pmatrix}
$$

In [8]:
def Rotation_theta(theta):
    return np.array([[np.cos(theta), np.sin(theta)],
                     [-np.sin(theta), np.cos(theta)]])

### Kinematic/Dynamic of the system

Positions :

$$
\begin{aligned}
    \overrightarrow{OT} &= x \  \overrightarrow{x_0} + y \  \overrightarrow{y_0} \\
    &= x_1 \  \overrightarrow{x_1} + y_1 \  \overrightarrow{y_1}
\end{aligned}
$$

Speeds :

$$
\begin{aligned}    
    \overrightarrow{V(T,1/0)} &= \left( \dot x_1 - \dot{\theta} y_1 \right) \  \overrightarrow{x_1} + \left(\dot y_1 + \dot \theta x_1 \right) \  \overrightarrow{y_1} \\
    \overrightarrow{V(I_d,1/0)} &= \left( \dot x_1 - \dot{\theta} y_1 - \dot \theta L \right) \  \overrightarrow{x_1} + \left(\dot y_1 + \dot \theta x_1 \right) \  \overrightarrow{y_1} \\
    \overrightarrow{V(I_g,1/0)} &= \left( \dot x_1 - \dot{\theta} y_1 + \dot \theta L \right) \  \overrightarrow{x_1} + \left(\dot y_1 + \dot \theta x_1 \right) \  \overrightarrow{y_1}
\end{aligned}
$$

The quantities that the robot measures and controls are the following :

$$
\left\{
\begin{aligned}
    V_{I_d} &= \overrightarrow{V(I_d,1/0)} . \overrightarrow{x_1} = \dot x_1 - \dot{\theta} y_1 - \dot \theta L \\
    V_{I_g} &= \overrightarrow{V(I_g,1/0)} . \overrightarrow{x_1} = \dot x_1 - \dot{\theta} y_1 + \dot \theta L
\end{aligned} \right. \Leftrightarrow
\left\{
\begin{aligned}
    \frac{V_{I_g} + V_{I_d}}{2} & = \dot x_1 - \dot{\theta} y_1 = \dot x \cos{\theta} + \dot y \sin{\theta}\\
    \frac{V_{I_g} - V_{I_d}}{2} &= \dot \theta L
\end{aligned} \right.
$$

### Dimensions of the parameters

To make sure that we don't have an interface problem, we paid attention to the units of the used quantities. We paid attention to the measurements of the Thymio which are in its own unit system. So we took a special care for them. We convert them.

- The coordinates $(x,y)$ are in mm
- The angle $(\theta)$ is in radian
- The speeds $(\dot x, \dot y)$ are in mm/s
- The angular speeds $(\dot \theta)$ are in rad/s



# Computer vision

# Path Finding

# Robot control

## Thymio class

**Talk about the thymio class**

## Control Algorithm

Based on the quantities that the thymio measures and control, the simplest is to separate the control in two parts. In a first time, the robot pivots to align with the direction it has to go. In a second time, it translates to a waypoint. Since the robot cannot align and reach the waypoint perfectly, we defined some margins for it that we will see later.

### Rotation control

In the particular case, $u = V_{I_g} = -V_{I_d}$, it is possible to show that the robot only rotates. By setting the state variable $z \in \mathbb{R}$ such that $z=\theta$. We get for the state equation :

$$
    \dot z = \frac{1}{L}u
$$

That can be discretized as :

$$
    z_{k+1} = z_k + \frac{T_s}{L} u_k
$$

We simply use a proportional controller $u_k = -K_{\text{rot}} z_k$ that verifies the following stability criteria :

$$
    \left| 1 - \frac{K_{\text{rot}} T_s}{L} \right| \leq 1 \Leftrightarrow 0 \leq K_{\text{rot}} \leq \frac{2 L}{T_s}
$$

The order sent to the wheel has a saturation :

$$
    u_k = \left\{\begin{aligned} &K_{\text{rot}} \left( \theta_b - \hat \theta_{k|k-1} \right) &\text{ if } \vert u_k \vert \leq u_{max} \\
    & u_{max} &\text{ if } u_k > u_{max} \\
    & -u_{max} &\text{ if } u_k < u_{max}
    \end{aligned}
    \right.
$$

with $\theta_b = \arctan_2{\left(\frac{y_f}{x_f}\right)}$ and $\hat \theta_{k|k-1}$ the predicted state computed by the Kalman filter.

In [ ]:
speedConversion = 0.797829 / 2.7
L = 46.75  # mm
Ts = 0.38
K_rotation = L/(8*Ts)

def rotation_control(self, theta_estimate, xb, yb):
    theta_b = np.arctan2(yb, xb)  # rad
    u = K_rotation * (theta_b - theta_estimate)  # mm/s
    u = u/speedConversion
    if u > 225:
        u = 225
    elif u < -225:
        u = -225
    left_motor_target = u  # in Thymio's unit
    right_motor_target = -u  # in Thymio's unit
    return left_motor_target, right_motor_target

### Translation control


In the particular case $u=V_{I_g}=V_{I_d}$, with the state variable $z=x_1$ the discrete equation is :

$$
    z_{k+1} = z_k + T_s u_k
$$

The proportional controller $u_k = -K_{\text{trans}} z_k$ verifies the following stability criteria :

$$
    0 \leq K_{\text{trans}} \leq \frac{2}{T_s}
$$

The order sent to the wheels is the following :

$$
    u_k = \left\{\begin{aligned} &K_{\text{trans}} \left( (x_1)_b - (\hat x_1)_{k|k-1} \right) &\text{ if } \vert u_k \vert \leq u_{max} \\
    & u_{max} &\text{ if } u_k > u_{max} \\
    & -u_{max} &\text{ if } u_k < u_{max}
    \end{aligned}
    \right.
$$

With $(x_1)_b = \cos \left(\theta_{k|k} \right) x_b + \sin \left( \theta_{k|k} \right) y_b$ and $(\hat x_1)_{k|k-1}$ the filtered state given by the Kalman filter



In [ ]:
K_translation = 1/(8*Ts)

def translation_control(pos_estimate, xb, yb):
    x_estimate = pos_estimate[0]  # mm
    y_estimate = pos_estimate[1]  # mm
    theta_estimate = pos_estimate[2]  # rad
    x1_b = np.cos(theta_estimate) * xb + np.sin(theta_estimate) * yb  # mm
    x1_estimate = np.cos(theta_estimate) * x_estimate + np.sin(theta_estimate) * y_estimate # mm
    u = K_translation * (x1_b - x1_estimate)  # en mm/s
    u = u / speedConversion
    if u > 225:
        u = 225
    elif u < -225:
        u = -225
    left_motor_target = u  # en unité Thymio
    right_motor_target = u  # en unité Thymio
    return left_motor_target, right_motor_target

### Margins control

So for the margins of the control algorithm we defined them as so :

- A square of half-length $l$ is set as the goal around the waypoint $(x_b,y_b)$
- A cone is defined from this square for which the robot should point to

The scheme below presents the problem :

![Image of the margins for the control](./img/marges_cone.jpeg)

#### Margin set waypoint

For numerical efficiency, the margin set around the waypoint $M_w$ can be written as a polytope.

$$
\begin{aligned}
    M_w &= \left\{ \begin{pmatrix}x\\ y \end{pmatrix} \in \mathbb{R}^2 : \vert x - x_b \vert \leq l, \vert y - y_b \vert \leq l   \right\} \\
    &= \left\{ \begin{pmatrix}x\\ y \end{pmatrix} \in \mathbb{R}^2 : \underbrace{\begin{pmatrix}1&0\\-1&0\\0&1\\0&-1 \end{pmatrix}}_{F} \begin{pmatrix}x-x_b\\ y-y_b \end{pmatrix} \leq l \ \mathbf{1} \right\}
\end{aligned}
$$

In [ ]:
l = 50 #mm

def robot_close_waypoint(pos_estimate, xb, yb):
        """_summary_
        Return a boolean to say if the robot is in a square with (xb,yb) as center and 2*l as length size
        Args:
            pos_estimate (1D np array with 2 variables): x,y position of the robot in mm
            xb (float): x position of the waypoint
            yb (float): y position of the waypoint
        """
        ones = np.array([1, 1, 1, 1])
        F = np.array([[1, 0], [-1, 0], [0, 1], [0, -1]])
        pos_waypoint = np.array([xb, yb])

        print("Is the goal reached ?")
        print(all(F @ (pos_estimate - pos_waypoint) <= l * ones))

        if all(F @ (pos_estimate - pos_waypoint) <= l * ones):
            return True
        else:
            return False

#### Margin set cone

The computation of the margin set for the yaw of the waypoint $C_w$ is a little bit more convolved. Based on the schema above, the set is defined as :

$$
    C_w = \left\{ \theta \in \mathbb{R} : \theta_{min} \leq \theta \leq \theta_{max} \right\}
$$

First, we have to compute the vertices of $M_w$. It is straightforward :

In [ ]:
def get_vertices_waypoint(xb, yb):
    vertices = np.array(
        [
            [xb + l, yb + l],
            [xb - l, yb + l],
            [xb - l, yb - l],
            [xb + l, yb - l],
        ]
    )
    return vertices

Since we know the position of the robot $(x,y)$ and the position of the vertices of $M_w$, let's call this set $V(M_w)$, we can easily get $\theta_{min}$ and $\theta_{max}$ by solving the following minimization-maximization problem :

$$
\left\{\begin{aligned}
&\theta_{max} = \max \Theta_w \\
&\theta_{min} = \min \Theta_w
\end{aligned}
\right.
$$

Where $\Theta_w$ is defined as :

$$
    \Theta_w = \left\{ \arctan_2 \left( \frac{y_v-y}{x_v-x} \right) : (x_v,y_v) \in V(M_w) \right\}
$$

To ensure that the robot is within the cone, we deine a margin $\gamma \in [0,0.5]$. Let $\delta = \theta_{max} - \theta_{min}$, $\hat \theta_{max} = \theta_{max} - \gamma \delta$ and $\hat \theta_{min} = \theta_{min} + \gamma \delta$. So we can set the margin as :

$$
\begin{aligned}
    \hat \delta &= \hat \theta_{max} - \hat \theta_{min} \\
                &= (1-2 \gamma) \delta
\end{aligned}
$$

In [1]:
coneMargin = 0.05

def get_cone_angles_waypoint(pos_estimate, xb, yb):
    """_summary_
    This function compute the range within which the robot should point before moving to the waypoint
    with a margin
    Args:
        pos_estimate (1D np array with 2 variables): x,y position of the robot in mm
        xb (float): x position of the waypoint
        yb (float): y position of the waypoint
        margin (float) : number between 0 and 1. (1-2*margin) corresponds to the coverage of the angle
        theta_max - theta_min
    """
    vertices = get_vertices_waypoint(xb, yb)
    angles = []
    for vertex in vertices:
        delta = vertex - pos_estimate
        angles.append(np.arctan2(delta[1], delta[0]))
    angles = np.array(angles)
    theta_max = np.max(angles)
    theta_min = np.min(angles)
    delta = theta_max - theta_min
    return theta_max - coneMargin * delta, theta_min + coneMargin * delta

def robot_align_waypoint(theta_estimate, theta_max, theta_min):
    if theta_estimate < theta_max and theta_estimate > theta_min:
        return True
    else:
        return False

# Kalman filter


The state vector $z_k \in \mathbb{R}^6$ is defined as so : $z_k = \left( x_k,y_k,\theta_k, \dot x_k, \dot y_k, \dot \theta_k \right)^T$. The equation of evolution is the following :

$$
\begin{aligned}
    z_{k+1} = \underbrace{\begin{pmatrix} I_3 & T_s I_3\\ 0 & I_3 \end{pmatrix}}_{A} &z_k + w \\ (s_c)_k = \underbrace{\begin{pmatrix}x_k\\y_k\\ \theta_k\\\dot x_k \cos{\theta_k} + \dot y_k \sin{\theta_k} \\ L \dot \theta_k\end{pmatrix}}_{g_c(z_k)} + v_c  \quad &\text{ou} \quad (s_{nc})_k = \underbrace{\begin{pmatrix}\dot x_k \cos{\theta_k} + \dot y_k \sin{\theta_k} \\ L \dot \theta_k\end{pmatrix}}_{g_{nc}(z_k)} + v_{nc}
\end{aligned}
$$

Where $w \sim \mathcal{N}(0,\,W)$, $v_c \sim \mathcal{N}(0,\,V_c)$, $v_{nc} \sim \mathcal{N}(0,\,V_{nc})$ and $z_0 \sim \mathcal{N}(\hat z_0,\,\Sigma_0)$ are all independant gaussian random vectors. 

In [ ]:
Ts = 1 #Define the sampling time before
L = 1 #Define before

W = np.identity(6)
V_c = np.identity(5)
V_nc = 20 # It is the same for V_theta and V_x_1

A = np.array([[1, 0, 0, Ts, 0, 0],
              [0, 1, 0, 0, Ts, 0],
              [0, 0, 1, 0, 0, Ts],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]])

def g_c(z):
    theta = z[2]
    s_c=[z[0], # x
        z[1], # y
        z[2], # theta
        z[3]*np.cos(theta) + z[4]*np.sin(theta), # x_dot*cos(theta) + y_dot*sin(theta)
        L*z[5]] # L*theta_dot
    return np.array(s_c)

The Kalman filter has to be able to switch between two states :

- Camera turned on 🎥 $\Longrightarrow$ position + wheels speed measured ($s_c$)
- Camera turned off 📏 $\Longrightarrow$ wheels speed measured ($s_{nc}$)

When the camera is turned off, it appears that the kalman filter works better to either reconstruct the translation or the rotation. Fortunately, the Thymio can only be in one of this two state. So we will further split the Kalman filter to the specific movements : $s_{nc} \Longrightarrow s_{\theta}, s_{x_1}$.

### $\theta$ dynamic

In rotation, the studied system is a reduced version of the one above. The parameters related to the translation ($x,y \dot x, \dot y$) are fixed in this case. So we will keep the values of those four parameters as the one of the latest predicted state before entering this reduced system.

$$
\begin{aligned}
    z_{\theta,k+1} = \begin{pmatrix}
        \theta_k \\ \dot \theta_k
    \end{pmatrix} &= \begin{pmatrix} 1 & T_s \\ 0 & 1 \end{pmatrix} z_{\theta,k} + w_{\theta}\\
    (s_{\theta,nc})_k &= \underbrace{\begin{pmatrix}
        0&L
    \end{pmatrix}}_{C_\theta} z_{\theta,k} + v_{\theta,nc} 
\end{aligned}
$$

Where $z_{\theta}$, $w_{\theta}$, $v_{\theta}$ are the reduced vectors of $z$, $w$ and $v_{nc}$ coming from their scalar entries on $\theta$ and $\dot \theta$.

In [ ]:
C_rot = np.array([[0, L]])

We will see later that not much will change in the philosophy of the Kalman equations with this reduced formulation. Nonetheless, we shall pay attention that the covariance matrix has also a reduced form as so :

$$
\Sigma_{\theta} = \begin{pmatrix}
        \Sigma_{\theta,\theta} & \Sigma_{\theta,\dot \theta}\\
        \Sigma_{\dot \theta,\theta} & \Sigma_{\dot \theta,\dot \theta}\\
    \end{pmatrix}
$$

Where $\Sigma$ is the covariance matrix of $z$ and $\Sigma_{\theta}$ the one of $z_{\theta}$. After the filtering step, we can put back the scalar entries of the reduced state vector and covariance matrix into the normal ones.

### $x_1$ dynamic

The reconstruction of the translation is more convolved. Looking at the quantities measured of the robot, it feels like that changing the coordinate system is better suited for this task as it will allow us to only reconstruct 2 variables. During translation, $\dot R(\theta) = 0$ so accoring to the change of basis, $\dot X_1 = R(\theta) \dot X_0$ thus :

$$    
z_1 = \begin{pmatrix}
        x_1\\y_1\\\theta\\\dot x_1\\ \dot y_1\\ \dot \theta
    \end{pmatrix} = \underbrace{\begin{pmatrix}
  \begin{matrix} R(\theta) & \begin{matrix} 0 \\ 0 \end{matrix} \\ \begin{matrix} 0 & 0 \end{matrix} & 1 \end{matrix} & \text{\huge 0} \\
  \text{\huge 0} & \begin{matrix} R(\theta) & \begin{matrix} 0 \\ 0 \end{matrix} \\ \begin{matrix} 0 & 0 \end{matrix} & 1 \end{matrix}
\end{pmatrix}}_{P_{1 \rightarrow 0}} z
 $$

Let's note that $P_{1 \rightarrow 0}$ is an orthogonal matrix and that $z = P_{0 \rightarrow 1} z_1 = P_{1 \rightarrow 0}^T z_1$. During translation, $\theta$ and $\dot \theta$ are fixed i.e. there are not random variables. So we can easily compute the covariance matrix of the random vector $z_1$ as so :

$$
\Sigma_1 = P_{1 \rightarrow 0} \Sigma P_{0 \rightarrow 1} \Longleftrightarrow \Sigma = P_{0 \rightarrow 1} \Sigma_1 P_{1 \rightarrow 0}
$$

In [ ]:
def P_1_vers_0(theta):
    R_theta = Rotation_theta(theta)
    return np.block([[R_theta, np.zeros((2,4))],
                        [0,0,1,0,0,0],
                        [np.zeros((2,3)), R_theta, np.zeros((2,1))],
                        [np.zeros((1,6))]])

def P_0_vers_1(theta):
    return P_1_vers_0(theta).T

We can finally write the equations of the reduced system :

$$
\begin{aligned}
    z_{x_1,k+1} = \begin{pmatrix}
        x_1 \\ \dot x_1
    \end{pmatrix} &= \begin{pmatrix} 1 & T_s \\ 0 & 1 \end{pmatrix} z_{x_1,k} + w_{x_1}\\
    (s_{x_1,nc})_k &= \underbrace{\begin{pmatrix}
        0&1
    \end{pmatrix}}_{C_{x_1}} z_{x_1,k} + v_{x_1,nc} 
\end{aligned}
$$

In [7]:
C_trans = np.array([[0, 1]])

And for the reduced covariance matrix :

$$
\Sigma_{x_1} = \begin{pmatrix}
        (\Sigma_1)_{x_1,x_1} & (\Sigma_1)_{x_1,\dot x_1}\\
        (\Sigma_1)_{\dot x_1,x_1} & (\Sigma_1)_{\dot x_1,\dot x_1}
    \end{pmatrix}
$$

Here again, the scalar entries of the reduced state vector and covariance matrix are put back into the main one and then $z$ is computed back :

$$
    z = P_{0 \rightarrow 1} z_1
$$

We introduce a boolean *translation_or_rotation* that is set to **True** if the robot is translating and to **False** if it is turning. We can now write $g_{nc}$.

In [ ]:
def g_nc(z, translation_or_rotation):
    if translation_or_rotation:
        return C_trans@z
    else:
        return C_rot@z

## Extended Kalman Filter 🤓

We are going to build an extended Kalman filter since the system is not linear. Let ($z_{k \mid k}, \Sigma_{k \mid k}$) the filtered state and ($z_{k+1 \mid k}, \Sigma_{k+1 \mid k}$) the predicted state.

First, let's compute the gradient of the deterministic function of the observed vectors :

$$
\begin{aligned}
    \nabla_{z} g_c(z) &= \begin{pmatrix}1&0&0&0&0&0\\
    0&1&0&0&0&0\\ 
    0&0&1&0&0&0\\ 
    0&0& -\dot x \sin{\theta} + \dot y \cos{\theta}& \cos{\theta} & \sin{\theta}&0\\ 0&0&0&0&0&L
    \end{pmatrix} \\
    \nabla_{z} g_{nc}(z) &= \left\{ \begin{aligned} &C_{\theta} \text{ If rotation}\\
    &C_{x_1} \text{ If translation}
    \end{aligned}\right.
\end{aligned}
$$

In [ ]:
def grad_g_c(z):
    theta = z[2]
    x_dot, y_dot = z[3], z[4]
    grad = [[1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0],
            [0, 0, -x_dot*np.sin(theta) + y_dot*np.cos(theta), np.cos(theta), np.sin(theta), 0],
            [0, 0, 0, 0, 0, L]]
    return np.array(grad)

def grad_g_nc(z, translation_or_rotation):
    if translation_or_rotation:
        return C_trans
    else:
        return C_rot

### Constructing the observed vector

Based on the state of the camera and the measures flowing into the code, we need a function to build the observed vector $s$.

In [ ]:
def constructing_s(V_left_measure, V_right_measure, camera_working=False,
                    x_measured=0, y_measured=0, theta_measured=0,                   
                    translation_or_rotation=True):
    if camera_working:
        s_c = np.array([x_measured,y_measured, theta_measured,
                        s_nc, (V_left_measure - V_right_measure)/2])
        return s_c
    elif translation_or_rotation:
        s_nc = (V_left_measure + V_right_measure)/2 # Mesure translation
    else:
        s_nc = (V_left_measure - V_right_measure)/2 # Mesure rotation
    return s_nc

### Prediction step - Camera on


$$
\begin{aligned}
\hat C_k &= \nabla_{z} g_c(\hat z_{k|k-1}) \\
V &= V_c\\

\Sigma_{k|k} & = \Sigma_{k|k-1}-\Sigma_{k|k-1}\hat{C}_{k}^{T}(\hat{C}_{k}\Sigma_{k|k-1} \hat{C}_{k}^{T}+V)^{-1} \hat{C}_{k} \Sigma_{k|k-1} \\

L_{k \mid k} &= \Sigma_{k \mid k-1}\hat{C}_{k}^{T}(\hat{C}_{k}\Sigma_{k|k-1}\hat{C}_{k}^{T}+V)^{-1}\\
    
\hat{z}_{k \mid k} &= \hat{z}_{k \mid k-1}+L_{k \mid k} \left((s_{c})_{k}-g_c(\hat{z}_{k \mid k-1})\right)
\end{aligned}
$$

### Prediction step - Camera off

$$
\begin{aligned}
\hat C_k &= \nabla_{z} g_{nc}(\hat z_{k|k-1}) \\
V &= \left\{\begin{aligned}&V_{\theta} \text{ If rotation}\\ &V_{x_1} \text{ If translation}\end{aligned}\right.\\

\Sigma_{k|k-1} &= \left\{\begin{aligned}&\Sigma_{k|k-1,\theta} \text{ If rotation}\\ &\Sigma_{k|k-1,x_1} \text{ If translation}\end{aligned}\right. \\

(s_{nc})_{k} &= \left\{\begin{aligned}& s_{\theta,nc} \text{ If rotation}\\ &s_{x_1,nc} \text{ If translation}\end{aligned}\right. \\ 

(g_{nc})_{k} &= \left\{\begin{aligned}& C_{\theta} \hat{z}_{\theta, k \mid k-1} \text{ If rotation}\\ &C_{x_1} \hat{z}_{x_1, k \mid k-1} \text{ If translation}\end{aligned}\right. \\ 


\Sigma_{k|k} & = \Sigma_{k|k-1}-\Sigma_{k|k-1}\hat{C}_{k}^{T}(\hat{C}_{k}\Sigma_{k|k-1} \hat{C}_{k}^{T}+V)^{-1} \hat{C}_{k} \Sigma_{k|k-1} \\

L_{k \mid k} &= \Sigma_{k \mid k-1}\hat{C}_{k}^{T}(\hat{C}_{k}\Sigma_{k|k-1}\hat{C}_{k}^{T}+V)^{-1}\\
    
\hat{z}_{k \mid k} &= \hat{z}_{k \mid k-1}+L_{k \mid k} \left((s_{nc})_{k}-(g_{nc})_{k}\right)
\end{aligned}
$$

And after the correct entire vector is reconstructed based on the reduced one.

In [ ]:
def filtering_step(z_k_k_1, sigma_k_k_1, V_left_measure, V_right_measure, 
                   camera_working=False, x_measured=0, y_measured=0,
                    theta_measured=0, translation_or_rotation=True):
    ### Computing the variables that are dependant on the state of the camera
    s_k = constructing_s(V_left_measure, V_right_measure, camera_working,
                          x_measured, y_measured, theta_measured, 
                          translation_or_rotation)
    C_k = grad_g_c(z_k_k_1) if camera_working else grad_g_nc(z_k_k_1, translation_or_rotation)
    V = V_c if camera_working else V_nc
    # Changing the computation of the filter based on the state of the camera
    ### The camera is on so all the states are reconstructed
    if camera_working:
        g_k = g_c(z_k_k_1)
        ### The filtering step that can be rewritten without any problem
        L_k_k = sigma_k_k_1@C_k.T@np.linalg.inv(C_k@sigma_k_k_1@C_k.T + V)
        sigma_k_k = sigma_k_k_1 - L_k_k@C_k@sigma_k_k_1
        z_k_k = z_k_k_1 + L_k_k@(s_k-g_k)

    ### The thymio is translating so only the translation is reconstructed
    elif translation_or_rotation:
        ### Changing the frame of coordinates
            theta = z_k_k_1[2]
            z_1_k_k_1 = P_1_vers_0(theta) @ z_k_k_1
            sigma_1_k_k_1 = P_1_vers_0(theta) @ sigma_k_k_1 @ P_0_vers_1(theta)

            ### Creating a reduced state vector and covariance matrix
            z_1_red_k_k_1 = np.array([z_1_k_k_1[0], z_1_k_k_1[3]])  # x1  & x1_dot
            sigma_1_red_k_k_1 = np.array(
                [
                    [sigma_1_k_k_1[0, 0], sigma_1_k_k_1[0, 3]],
                    [sigma_1_k_k_1[3, 0], sigma_1_k_k_1[3, 3]],
                ]
            )

            g_k = g_nc(z_1_red_k_k_1, translation_or_rotation) 

            ### The filtering step can be computed for the reduced system
            L_red_k_k = (
                sigma_1_red_k_k_1
                @ C_k.T
                @ np.linalg.inv(C_k @ sigma_1_red_k_k_1 @ C_k.T + V)
            )
            sigma_1_red_k_k = sigma_1_red_k_k_1 - L_red_k_k @ C_k @ sigma_1_red_k_k_1
            z_1_red_k_k = z_1_red_k_k_1 + L_red_k_k @ (s_k - g_k)

            ### Putting back the reduced vector into the main one as well as the covariance matrix
            z_1_k_k = z_1_k_k_1
            z_1_k_k[0], z_1_k_k[3] = z_1_red_k_k[0], z_1_red_k_k[1]

            sigma_1_k_k = sigma_1_k_k_1
            sigma_1_k_k[0, 0], sigma_1_k_k[0, 3] = sigma_1_red_k_k[0, 0], sigma_1_red_k_k[0, 1]
            sigma_1_k_k[3, 0], sigma_1_k_k[3, 3] = sigma_1_red_k_k[1, 0], sigma_1_red_k_k[1, 1]

            ### Going back to the original frame
            z_k_k = P_0_vers_1(theta) @ z_1_k_k
            sigma_k_k = P_0_vers_1(theta) @ sigma_1_k_k @ P_1_vers_0(theta)

    ### The thyimio is turning so only the rotation is recontructed
    else:
        ### Extracting theta and theta_dot
        z_red_k_k_1 = np.array([z_k_k_1[2],
                                z_k_k_1[5]])
        
        sigma_red_k_k_1 = np.array([[sigma_k_k_1[2,2], sigma_k_k_1[2,5]],
                                    [sigma_k_k_1[5,2], sigma_k_k_1[5,5]]])
        
        g_k = g_nc(z_red_k_k_1, translation_or_rotation)
        ### The filtering step can be computed for the reduced system
        L_red_k_k = sigma_red_k_k_1@C_k.T@np.linalg.inv(C_k@sigma_red_k_k_1@C_k.T + V)
        sigma_red_k_k = sigma_red_k_k_1 - L_red_k_k@C_k@sigma_red_k_k_1
        z_red_k_k = z_red_k_k_1 + L_red_k_k@(s_k-g_k)

        z_k_k = z_k_k_1
        z_k_k[2], z_k_k[5] = z_red_k_k[0], z_red_k_k[1]

        sigma_k_k = sigma_k_k_1
        sigma_k_k[2,2], sigma_k_k[2,5] = sigma_red_k_k[0,0], sigma_red_k_k[0,1]
        sigma_k_k[5,2], sigma_k_k[5,5] = sigma_red_k_k[1,0], sigma_red_k_k[1,1]
        

    return z_k_k, sigma_k_k

### Prediction step

The filtering step is unchanged in all cases. It is straighforward to write it down : 

$$
\begin{aligned}
    \hat z_{k+1|k} &= A \hat z_{k|k}\\
    \Sigma_{k+1|k} &= A \Sigma_{k|k} A^T + W
\end{aligned}
$$

In [ ]:
def prediction_step(z_k_k, sigma_k_k):
    z_k_1_k = A@z_k_k
    sigma_k_1_k = A@sigma_k_k@A.T + W
    return z_k_1_k, sigma_k_1_k

### Simulating the Kalman filter

# Running the project ?

# Conclusion